In [1]:
from qbuki import *

# qbuki.optimize

`qbuki.optimize` provides tools for searching out POVM's (and more generally, QBist reference devices) that extremize some useful measure. At the moment, it focuses primarily on the so-called quantumness: $|| I - \Phi||$.

The central actor is:

```python
min_quantumness(d, n=None, field="complex",
                           norm="p2",\
                           rank1=True,\
                           parallel=True,\
                           method="SLSQP",\
                           tol=1e-26,\
                           options={"disp": False,\
                                    "maxiter": 10000},\
                           max_iter=100,\
                           return_params=False)
```
One provides the Hilbert space dimension $d$; the number of POVM elements $n$ (defaulting to $d^2$ in the complex case and $d(d+1)/2$ in the real case); the number field (`real` or `complex`); the norm by which to measure the distance between $\Phi$ and the identity, which defaults to the Schatten 2-norm, but one can specify another p-norm, e.g. "p1" or indeed, pass in an arbitrary norm function; whether to restrict the search to rank-1 elements, which leads to substantial savings in efficiency; whether to take post-measurement states to be proportional to POVM elements or whether to search over them separately; the method used by the optimizer (`scipy.optimize.minimize`) and options to be passed to the optimizer; the number of iterations to attempt running the optimizer; and finally, whether to return the underlying parameterization of the reference device instead of POVM elements (and/or post-measurement states): for a POVM, in the case of rank-1, this would be a matrix of frame vectors, and in the higher rank case, this would be a matrix of Kraus operators (and the analogous for the post-measurement states.)

For example, we can find the SIC-POVM in real $d=2$:

In [6]:
E = Operators(min_quantumness(2, n=3, field="real", norm="p2", rank1=True, parallel=True))
E^E

array([[0.444, 0.111, 0.111],
       [0.111, 0.444, 0.111],
       [0.111, 0.111, 0.444]])

In [10]:
R = min_quantumness(2, n=3, field="real", norm="p2", rank1=True, parallel=True, return_params=True); R

array([[-0.702,  0.01 ,  0.712],
       [-0.417, -0.816, -0.4  ]])

In [11]:
R @ R.T

array([[1., 0.],
       [0., 1.]])

In complex $d=2$:

In [12]:
E = Operators(min_quantumness(2, n=4, field="complex", norm="p2", rank1=True, parallel=True))
E^E

array([[0.25 +0.j, 0.083-0.j, 0.083+0.j, 0.083-0.j],
       [0.083+0.j, 0.25 +0.j, 0.083-0.j, 0.083+0.j],
       [0.083-0.j, 0.083+0.j, 0.25 +0.j, 0.083+0.j],
       [0.083+0.j, 0.083-0.j, 0.083-0.j, 0.25 +0.j]])

We can try a higher rank search:

In [13]:
E = Operators(min_quantumness(2, n=4, field="complex", norm="p2", rank1=False, parallel=True))
E^E

array([[0.25 +0.j, 0.083-0.j, 0.083-0.j, 0.083-0.j],
       [0.083+0.j, 0.25 +0.j, 0.083-0.j, 0.083-0.j],
       [0.083+0.j, 0.083+0.j, 0.25 +0.j, 0.083+0.j],
       [0.083+0.j, 0.083+0.j, 0.083-0.j, 0.25 +0.j]])

And search over both POVM elements and post-measurement states

In [68]:
E, S = min_quantumness(2, field="complex", norm="p1", rank1=True, parallel=False, method="trust-constr")
E, S = Operators(E), Operators(S)

In [69]:
quantumness(E, S, p=1)

6.001083834026661

In [70]:
H = Operators(min_quantumness(2, field="complex", norm="p1", rank1=True, parallel=True, method="trust-constr"))

In [71]:
quantumness(H,p=1)

6.00000524922556